# 数据处理

## 数据处理 - 吸烟

smoking.xlsx

> col 0	            6位数字ID，前两位代表“县”  
> col 1和col 2合起来   表示地点  
> col 3	            烟税
> 
> 需要计算当地的人均烟税额，和该县的人均烟税额


FEH_00200251_190808113903

> 日本的烟税分三层收三次：国税、都道府县税、市町村税。经过调查，都道府县烟税全部为0，国税烟税只有各地国税局级别的数据，市町村烟税有高精度数据。

> FEH_00200251_190808113903 系地方税，各地自行制定税率，实际上不能横向比较。


In [136]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import numpy as np
import pandas as pd
pd.set_option('display.width', 200)   # 每行最大字符
pd.set_option('precision', 3)         # 显示数字精度
pd.set_option('display.max_rows', 10) # 预览时最多显示行数
pd.set_option('display.float_format', lambda x : '%.2f' % x)  # 不使用科学计数法

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.transforms as transforms

plt.rcParams['figure.figsize'] = 18, 9
plt.rcParams['axes.unicode_minus'] = False     # 显示数字负号
plt.rcParams['font.sans-serif'] = ['SimHei']   # 显示中文字体
mpl.rcParams['figure.dpi'] = 80
mpl.rcParams['savefig.dpi'] = 100
mpl.rcParams['font.size'] = 12
mpl.rcParams['legend.fontsize'] = 'large'
mpl.rcParams['figure.titlesize'] = 'medium'
plt.style.use('seaborn-whitegrid')

import seaborn as sns

def explore_df(df):
  print('==== row, col ====', df.shape)
  print('==== columns ====')
  print('\t'.join(df.columns))

  print('==== describe ====')
  print(df.describe())
  print('==== info ====')
  print(df.info())
  print('==== sample ====')
  return df.sample(n=5, random_state=100)
  
# explore_df(df)

def find_in_df(df, col, pat):
  ''' 在 col 中 查找命中 pat 的数据
      如果 pat 是 字符串, 视为模糊查找 
      如果 pat 是 list, 或者非字符串型, 视为精确查找 '''
  if isinstance(pat, str):
    return df[df[col].str.contains(pat)]
  if not isinstance(pat, (list, set)):
    pat = [pat]
  return df[df[col].isin(pat)]



### 预览 smoking.xlsx

In [137]:
df = pd.read_excel("../smoking.xlsx", sheet_name="Sheet1")

In [138]:

explore_df(df)

==== row, col ==== (1741, 4)
==== columns ====
団体
コード	Unnamed: 1	団体名	市町村たばこ税
==== describe ====
        団体\nコード
count   1741.00
mean  211462.81
std   143348.78
min    11002.00
25%    82341.00
50%   204030.00
75%   332038.00
max   473821.00
==== info ====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1741 entries, 0 to 1740
Data columns (total 4 columns):
団体
コード        1741 non-null int64
Unnamed: 1    1741 non-null object
団体名           1741 non-null object
市町村たばこ税       1741 non-null object
dtypes: int64(1), object(3)
memory usage: 54.5+ KB
None
==== sample ====


,団体 コード,Unnamed: 1,団体名,市町村たばこ税
1688,465054,鹿児島県,屋久島町,100915
1198,292036,奈良県,大和郡山市,106924
1224,294411,奈良県,吉野町,40171
1512,405035,福岡県,大刀洗町,140541
27,12289,北海道,深川市,5849


其中, `団体コード` ID 可能是六位数, 因为

> 头2位是JIS X 0401制定的都道府县编号，加上后3位共5位为JIS X 0402的市区町村编号及一部事务组合等编号，最后1位是检查数字（校验码）。有些时候只使用5位编号而不记录检查数字。

### 预览 FEH_00200251_190808113903.csv

In [139]:
smoking_df = pd.read_csv("../smoking/FEH_00200251_190808113903.csv", encoding='shift-jis', header=7)

In [140]:
explore_df(smoking_df)

==== row, col ==== (100884, 11)
==== columns ====
表章項目 コード	表章項目	時間軸(年度次) コード	時間軸(年度次)	団体名(市町村分) コード	団体名(市町村分)	市町村税 コード	市町村税	/徴収課税	調定済額_現年課税分(A)	注釈
==== describe ====
       表章項目 コード  時間軸(年度次) コード  団体名(市町村分) コード  市町村税 コード  /徴収課税   注釈
count 100884.00     100884.00      100884.00 100884.00   0.00 0.00
mean  100100.00 2002600000.00       23273.70    280.00    nan  nan
std        0.00    8077787.25       14140.51      0.00    nan  nan
min   100100.00 1989100000.00        1100.00    280.00    nan  nan
25%   100100.00 1995850000.00       11205.00    280.00    nan  nan
50%   100100.00 2002600000.00       22422.00    280.00    nan  nan
75%   100100.00 2009350000.00       35443.00    280.00    nan  nan
max   100100.00 2016100000.00       47382.00    280.00    nan  nan
==== info ====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100884 entries, 0 to 100883
Data columns (total 11 columns):
表章項目 コード         100884 non-null int64
表章項目             100884 non-null object
時間軸(年度次) コード     100884 no

,表章項目 コード,表章項目,時間軸(年度次) コード,時間軸(年度次),団体名(市町村分) コード,団体名(市町村分),市町村税 コード,市町村税,/徴収課税,調定済額_現年課税分(A),注釈
52607,100100,金額【千円】,2002100000,2002年度,28207,伊丹市,280,一.普通税_1.法定普通税_(4)市町村たばこ税,nan,"1,167,055",nan
66433,100100,金額【千円】,1998100000,1998年度,20414,泰阜村,280,一.普通税_1.法定普通税_(4)市町村たばこ税,nan,"4,109",nan
58424,100100,金額【千円】,2000100000,2000年度,9210,大田原市,280,一.普通税_1.法定普通税_(4)市町村たばこ税,nan,"415,555",nan
70291,100100,金額【千円】,1997100000,1997年度,23203,一宮市,280,一.普通税_1.法定普通税_(4)市町村たばこ税,nan,"1,531,528",nan
29027,100100,金額【千円】,2008100000,2008年度,1643,幕別町,280,一.普通税_1.法定普通税_(4)市町村たばこ税,nan,"158,140",nan


In [141]:
# 保留所需columns, 重命名

column_map = [
  ('時間軸(年度次)',           '年度次'),
  ('団体名(市町村分) コード',  'ID'),      
  ('団体名(市町村分)',         '市町村'),
  ('調定済額_現年課税分(A)',    '現年課税'),  
]

smoking_df = smoking_df[[line[0] for line in column_map]]
smoking_df.columns = [line[1] for line in column_map]
smoking_df.head(5)

,年度次,ID,市町村,現年課税
0,2016年度,1100,札幌市,"15,793,662"
1,2016年度,1202,函館市,"2,502,233"
2,2016年度,1203,小樽市,"1,010,892"
3,2016年度,1204,旭川市,"2,977,616"
4,2016年度,1205,室蘭市,"767,900"


In [142]:
def convert_num(s):
  if s == '***':
    return np.nan
  return int(s.replace(',', ''))

# 将 '現年課税' 中的 '***' 转为 NaN
smoking_df['現年課税'] = smoking_df['現年課税'].map(convert_num)


# 总计多少无效 rows
smoking_df[smoking_df[['現年課税']].applymap(pd.isna).all(1)]

,年度次,ID,市町村,現年課税
35,2016年度,1301,広島町,nan
36,2016年度,1302,石狩町,nan
39,2016年度,1305,厚田村,nan
40,2016年度,1306,浜益村,nan
45,2016年度,1335,上磯町,nan
...,...,...,...,...
100836,1989年度,47213,うるま市,nan
100837,1989年度,47214,宮古島市,nan
100838,1989年度,47215,南城市,nan
100875,1989年度,47361,久米島町,nan


In [143]:
# 重新整理 `ID`

def convert_id(i):
  # 1203  => "01-203"
  s = str(int(i))
  if i < 10000: s = "0" + s
  return s[:2] + '-' + s[2:]

smoking_df['ID'] = smoking_df['ID'].map(convert_id)
explore_df(smoking_df)

==== row, col ==== (100884, 4)
==== columns ====
年度次	ID	市町村	現年課税
==== describe ====
             現年課税
count    72504.00
mean    308857.88
std    1076195.51
min          0.00
25%      27470.00
50%      66278.00
75%     213935.50
max   32134416.00
==== info ====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100884 entries, 0 to 100883
Data columns (total 4 columns):
年度次     100884 non-null object
ID      100884 non-null object
市町村     100884 non-null object
現年課税    72504 non-null float64
dtypes: float64(1), object(3)
memory usage: 3.1+ MB
None
==== sample ====


,年度次,ID,市町村,現年課税
52607,2002年度,28-207,伊丹市,1167055.00
66433,1998年度,20-414,泰阜村,4109.00
58424,2000年度,09-210,大田原市,415555.00
70291,1997年度,23-203,一宮市,1531528.00
29027,2008年度,01-643,幕別町,158140.00


In [144]:
# 按照年度分组, 统计每年有多少有效数据, 多少无效数据

for index, data in smoking_df.groupby('年度次'):
  print(f'{index}: ')
  print(data['現年課税'].map(lambda x: 'NaN' if pd.isna(x) else 'Valid').value_counts())

# 早期年份的有效数据更多??

1989年度: 
Valid    3268
NaN       335
Name: 現年課税, dtype: int64
1990年度: 
Valid    3264
NaN       339
Name: 現年課税, dtype: int64
1991年度: 
Valid    3260
NaN       343
Name: 現年課税, dtype: int64
1992年度: 
Valid    3259
NaN       344
Name: 現年課税, dtype: int64
1993年度: 
Valid    3258
NaN       345
Name: 現年課税, dtype: int64
1994年度: 
Valid    3257
NaN       346
Name: 現年課税, dtype: int64
1995年度: 
Valid    3255
NaN       348
Name: 現年課税, dtype: int64
1996年度: 
Valid    3255
NaN       348
Name: 現年課税, dtype: int64
1997年度: 
Valid    3255
NaN       348
Name: 現年課税, dtype: int64
1998年度: 
Valid    3255
NaN       348
Name: 現年課税, dtype: int64
1999年度: 
Valid    3252
NaN       351
Name: 現年課税, dtype: int64
2000年度: 
Valid    3250
NaN       353
Name: 現年課税, dtype: int64
2001年度: 
Valid    3246
NaN       357
Name: 現年課税, dtype: int64
2002年度: 
Valid    3235
NaN       368
Name: 現年課税, dtype: int64
2003年度: 
Valid    3155
NaN       448
Name: 現年課税, dtype: int64
2004年度: 
Valid    2544
NaN      1059
Name: 現年課税, dtype: int64
2005年度: 

### 引入人口数据, 计算人均烟税

In [145]:
population_df = pd.read_csv("人口.csv", encoding='utf-8')
explore_df(population_df)

==== row, col ==== (13928, 8)
==== columns ====
Unnamed: 0	调查年份	地域数字ID	都道府县	市町村	人口数量	人口数量男	人口数量女
==== describe ====
       Unnamed: 0       人口数量      人口数量男      人口数量女
count    13928.00   13928.00    6964.00    6964.00
mean      7658.32   71591.50   35701.84   37509.35
std       4426.34  175268.83   89846.94   93271.89
min          0.00       0.00       0.00       0.00
25%       3831.75    9689.50    4344.00    4658.75
50%       7663.50   26391.50   12491.00   13543.00
75%      11495.25   64207.75   31805.00   33567.00
max      15327.00 3724844.00 1855985.00 1868859.00
==== info ====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13928 entries, 0 to 13927
Data columns (total 8 columns):
Unnamed: 0    13928 non-null int64
调查年份          13928 non-null object
地域数字ID        13928 non-null object
都道府县          13928 non-null object
市町村           13928 non-null object
人口数量          13928 non-null int64
人口数量男         6964 non-null float64
人口数量女         6964 non-null float64
dtypes: float64(

,Unnamed: 0,调查年份,地域数字ID,都道府县,市町村,人口数量,人口数量男,人口数量女
6139,6731,2000年度,21-210,岐阜県,恵那市,57274,27785.00,29489.00
5694,6234,2000年度,09-384,栃木県,塩谷町,14171,6876.00,7295.00
8582,9457,1995年度,44-206,大分県,臼杵市,46830,nan,nan
2287,2487,2010年度,11-242,埼玉県,日高市,57473,28548.00,28925.00
1661,1836,2015年度,46-208,鹿児島県,出水市,53758,24984.00,28774.00


In [146]:
# smoking 和 population 都做出 `年份:XX-YYY` 的临时字段
smoking_df['yearlocation'] = smoking_df['年度次'].map(lambda s: s[:4]) + ':' + smoking_df['ID']
smoking_df

,年度次,ID,市町村,現年課税,yearlocation
0,2016年度,01-100,札幌市,15793662.00,2016:01-100
1,2016年度,01-202,函館市,2502233.00,2016:01-202
2,2016年度,01-203,小樽市,1010892.00,2016:01-203
3,2016年度,01-204,旭川市,2977616.00,2016:01-204
4,2016年度,01-205,室蘭市,767900.00,2016:01-205
...,...,...,...,...,...
100879,1989年度,47-373,上野村,10114.00,1989:47-373
100880,1989年度,47-374,伊良部町,25341.00,1989:47-374
100881,1989年度,47-375,多良間村,4481.00,1989:47-375
100882,1989年度,47-381,竹富町,11305.00,1989:47-381


In [147]:
# smoking 和 population 都做出 `年份:XX-YYY` 的临时字段
population_df['yearlocation'] = population_df['调查年份'].map(lambda s: s[:4]) + ':' + population_df['地域数字ID']
population_df

,Unnamed: 0,调查年份,地域数字ID,都道府县,市町村,人口数量,人口数量男,人口数量女,yearlocation
0,0,2015年度,01-100,北海道,札幌市,1952356,910614.00,1041742.00,2015:01-100
1,11,2015年度,01-202,北海道,函館市,265979,120376.00,145603.00,2015:01-202
2,12,2015年度,01-203,北海道,小樽市,121924,54985.00,66939.00,2015:01-203
3,13,2015年度,01-204,北海道,旭川市,339605,156402.00,183203.00,2015:01-204
4,14,2015年度,01-205,北海道,室蘭市,88564,43143.00,45421.00,2015:01-205
...,...,...,...,...,...,...,...,...,...
13923,15323,1980年度,47-361,沖縄県,久米島町,10187,nan,nan,1980:47-361
13924,15324,1980年度,47-362,沖縄県,八重瀬町,18990,nan,nan,1980:47-362
13925,15325,1980年度,47-375,沖縄県,多良間村,1667,nan,nan,1980:47-375
13926,15326,1980年度,47-381,沖縄県,竹富町,3376,nan,nan,1980:47-381


In [148]:
# 以 `yearlocation` 为外键连接 smoking <- population
# 使用左连接
# 先只连接 2015年

smoking_df = smoking_df[smoking_df.年度次 == '2015年度']
smoking_df_merge_pop = pd.merge(smoking_df, population_df, how='left', on='yearlocation', 
                                left_index=False, right_index=False, sort=True,  
                                suffixes=('', '_from_pop'), copy=True, indicator=False)

In [149]:
print('预览 smoking_df_merge_pop')
smoking_df_merge_pop

print('在 smoking 中, 但不在 population 中的地点')
smoking_df_merge_pop[smoking_df_merge_pop.地域数字ID.isna()]

预览 smoking_df_merge_pop


,年度次,ID,市町村,現年課税,yearlocation,Unnamed: 0,调查年份,地域数字ID,都道府县,市町村_from_pop,人口数量,人口数量男,人口数量女
0,2015年度,01-100,札幌市,16200206.00,2015:01-100,0.00,2015年度,01-100,北海道,札幌市,1952356.00,910614.00,1041742.00
1,2015年度,01-202,函館市,2566741.00,2015:01-202,11.00,2015年度,01-202,北海道,函館市,265979.00,120376.00,145603.00
2,2015年度,01-203,小樽市,1045317.00,2015:01-203,12.00,2015年度,01-203,北海道,小樽市,121924.00,54985.00,66939.00
3,2015年度,01-204,旭川市,3081043.00,2015:01-204,13.00,2015年度,01-204,北海道,旭川市,339605.00,156402.00,183203.00
4,2015年度,01-205,室蘭市,798421.00,2015:01-205,14.00,2015年度,01-205,北海道,室蘭市,88564.00,43143.00,45421.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3598,2015年度,47-373,上野村,nan,2015:47-373,nan,NaN,NaN,NaN,NaN,nan,nan,nan
3599,2015年度,47-374,伊良部町,nan,2015:47-374,nan,NaN,NaN,NaN,NaN,nan,nan,nan
3600,2015年度,47-375,多良間村,6511.00,2015:47-375,1913.00,2015年度,47-375,沖縄県,多良間村,1194.00,639.00,555.00
3601,2015年度,47-381,竹富町,22125.00,2015:47-381,1914.00,2015年度,47-381,沖縄県,竹富町,3998.00,2057.00,1941.00


在 smoking 中, 但不在 population 中的地点


,年度次,ID,市町村,現年課税,yearlocation,Unnamed: 0,调查年份,地域数字ID,都道府县,市町村_from_pop,人口数量,人口数量男,人口数量女
35,2015年度,01-301,広島町,nan,2015:01-301,nan,NaN,NaN,NaN,NaN,nan,nan,nan
36,2015年度,01-302,石狩町,nan,2015:01-302,nan,NaN,NaN,NaN,NaN,nan,nan,nan
39,2015年度,01-305,厚田村,nan,2015:01-305,nan,NaN,NaN,NaN,NaN,nan,nan,nan
40,2015年度,01-306,浜益村,nan,2015:01-306,nan,NaN,NaN,NaN,NaN,nan,nan,nan
45,2015年度,01-335,上磯町,nan,2015:01-335,nan,NaN,NaN,NaN,NaN,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3585,2015年度,47-352,具志川村,nan,2015:47-352,nan,NaN,NaN,NaN,NaN,nan,nan,nan
3596,2015年度,47-371,城辺町,nan,2015:47-371,nan,NaN,NaN,NaN,NaN,nan,nan,nan
3597,2015年度,47-372,下地町,nan,2015:47-372,nan,NaN,NaN,NaN,NaN,nan,nan,nan
3598,2015年度,47-373,上野村,nan,2015:47-373,nan,NaN,NaN,NaN,NaN,nan,nan,nan


In [153]:
# 在 smoking 中, 但不在 population 中的地点是 1862 行
# 而之前在 smoking 2015年中, 也是恰好有 1862 行 NaN 数据

# 但是两者并不一致, 有一条富谷町数据有烟税, 却在人口中找不到
# 不管它
temp = smoking_df_merge_pop[smoking_df_merge_pop.地域数字ID.isna()]
temp[~temp.現年課税.isna()]

,年度次,ID,市町村,現年課税,yearlocation,Unnamed: 0,调查年份,地域数字ID,都道府县,市町村_from_pop,人口数量,人口数量男,人口数量女
392,2015年度,04-423,富谷町,300326.00,2015:04-423,nan,NaN,NaN,NaN,NaN,nan,nan,nan


In [156]:
# 计算人均烟税
smoking_df_merge_pop['人均烟税'] = smoking_df_merge_pop.現年課税 / smoking_df_merge_pop.人口数量

# 去除无用字段
columns = ['年度次', 'ID', '都道府县', '市町村', '現年課税', '人口数量', '人均烟税']
smoking_df_merge_pop = smoking_df_merge_pop[columns]
smoking_df_merge_pop

,年度次,ID,都道府县,市町村,現年課税,人口数量,人均烟税
0,2015年度,01-100,北海道,札幌市,16200206.00,1952356.00,8.30
1,2015年度,01-202,北海道,函館市,2566741.00,265979.00,9.65
2,2015年度,01-203,北海道,小樽市,1045317.00,121924.00,8.57
3,2015年度,01-204,北海道,旭川市,3081043.00,339605.00,9.07
4,2015年度,01-205,北海道,室蘭市,798421.00,88564.00,9.02
...,...,...,...,...,...,...,...
3598,2015年度,47-373,NaN,上野村,nan,nan,nan
3599,2015年度,47-374,NaN,伊良部町,nan,nan,nan
3600,2015年度,47-375,沖縄県,多良間村,6511.00,1194.00,5.45
3601,2015年度,47-381,沖縄県,竹富町,22125.00,3998.00,5.53


In [158]:
print(smoking_df_merge_pop.to_csv(), file=open('人均烟税.csv', 'w'))